In [1]:
import uuid
from typing import Tuple, List, Dict, Any, Optional
from dataclasses import dataclass, field
from functools import reduce
from rdflib import Graph as RDFGraph

NTuple = Tuple[Any, ...]

@dataclass
class Graph:
    triples: List[Tuple[Any, ...]] = field(default_factory=list)
    
    def append(self, nt: NTuple):
        self.triples.append(nt)
        
    def match(self, s = None, p = None, o = None):        
        return [t for t in self.triples if 
                (not s or t[0] == s) 
                and (not p or t[1] == p)
                and (not o or t[2] == o)]
    def parse(self, uri, fmt='ttl'):
        g = RDFGraph().parse(uri, format=fmt)
        for s, p, o in g.triples((None, None, None)):
            self.append((s, p, o))

In [2]:
class Infix:
    def __init__(self, function):
        self.function = function
    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __or__(self, other):
        return self.function(other)
    def __rlshift__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __rshift__(self, other):
        return self.function(other)
    def __call__(self, value1, value2):
        return self.function(value1, value2)


In [3]:
#rdfs
g = Graph()

a = Infix(lambda s, o: g.append((s,'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',o)))

enemyOf = Infix(lambda s, o: g.append((s, ':enemyOf', o)))

class Foaf:
    Person = 'http://xmlns.com/foaf/0.1/Person'
    name = Infix(lambda s, o: g.append((s, 'http://xmlns.com/foaf/0.1/name', o)))
    mbox = Infix(lambda s, o: g.append((s, 'http://xmlns.com/foaf/0.1/mbox', o)))
    homepage = Infix(lambda s, o: g.append((s, 'http://xmlns.com/foaf/0.1/homepage', o)))

In [4]:
':green-goblin' |enemyOf|   ':spiderman'
':green-goblin' |a|         Foaf.Person
':green-goblin' |Foaf.name| 'Green Goblin'
':green-goblin' |Foaf.mbox| 'Norman.Osborn@marvel.com'

In [5]:
for s, p, o in g.match(':green-goblin'):
    print (s, p, o)

:green-goblin :enemyOf :spiderman
:green-goblin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
:green-goblin http://xmlns.com/foaf/0.1/name Green Goblin
:green-goblin http://xmlns.com/foaf/0.1/mbox Norman.Osborn@marvel.com


In [6]:
g.parse('https://raw.githubusercontent.com/emmo-repo/domain-mappings/master/mappings.ttl')

In [7]:
for s, p, o in g.match():
    print (s, p, o)

:green-goblin :enemyOf :spiderman
:green-goblin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
:green-goblin http://xmlns.com/foaf/0.1/name Green Goblin
:green-goblin http://xmlns.com/foaf/0.1/mbox Norman.Osborn@marvel.com
http://emmo.info/domain-mappings#Domain http://www.w3.org/2000/01/rdf-schema#subClassOf n2cbae0674ee04d7899048259ddccc214b1
http://emmo.info/domain-mappings#hasIRI http://www.w3.org/2000/01/rdf-schema#domain http://emmo.info/domain-mappings#OntologicalConcept
http://emmo.info/domain-mappings#OntologicalConcept http://emmo.info/emmo#EMMO_967080e5_2f42_4eb2_a3a9_c58143e835f9 An entity (class, individual or property) in an ontology.
http://emmo.info/domain-mappings#PROMO_0293b6d0_344d_430a_a572_f196ae5bc051 http://www.w3.org/2000/01/rdf-schema#label crystallography_ontology
http://emmo.info/domain-mappings#hasIRI http://emmo.info/emmo#EMMO_967080e5_2f42_4eb2_a3a9_c58143e835f9 The IRI of an ontological concept.
http://emmo.info/domain-ma